# Precision calculations for Similarity Experiment 2 (covering three tune families in full)

# Import dependencies:

In [1]:
import glob
import matplotlib.pyplot as plt
import ml_metrics
import os

import numpy as np
import pandas as pd

# Set target tune families and list their similarity results csv files, which are to be used as input data for precision calculations:

In [2]:
# List all tune folders in tune family:
root_dir = "/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/"
tune_families = [path.name for path in os.scandir(root_dir) if os.path.isdir(path)]
print(tune_families)
concatenated_inpaths = {fam: None for fam in tune_families}
for fam in tune_families:
    input_filepaths = []
    target_dir = root_dir + fam
    fam_subdirectories = glob.glob(f"{target_dir}/*/", recursive = True)
    # for each tune family, create list of lists, per:
    # top level list holds all tune folders
    # each lower-level list holds the filepaths within an individual tune folder, as long as they match the filter below.
    for dir in fam_subdirectories:
        filepaths = [path for path in glob.iglob(os.path.join(dir, "**", "*.csv"), recursive=True) if not "patterns" in path or 'precision' in path]
        input_filepaths.append(filepaths)

    # create flattened version of the nested lists:
    flattened_tune_fam_input_filepaths = [path for subdir in input_filepaths for path in subdir]
    concatenated_inpaths[fam] = flattened_tune_fam_input_filepaths

    # Print all csv filepaths, i.e. the entire flattened list:

print("Input filepaths:")
print(concatenated_inpaths)



['jennys_welcome_to_charlie.csv', 'drowsy_maggie.csv', 'gilderoy.csv', 'greigs_pipes.csv', 'blackbird.csv', 'hob_or_nob.csv', 'osullivans_march.csv', 'road_to_lisdoonvarna.csv', 'lord_mcdonalds.csv', 'johnny_cope.csv']
Input filepaths:
{'jennys_welcome_to_charlie.csv': ['/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/structural_results/incipit__lev.csv', '/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/structural_results/incipit_cadence_cos.csv', '/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/structural_results/incipit_cadence_lev.csv', '/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/structural_results/incipit__cos.csv', '/Users/dannydiamon

# Identify and extract ground truth data for all tune families:

In [3]:
# base path to The Session ground truth dir:
grd_truth_basepath = '/Users/dannydiamond/NUIG/Polifonia/thesession/test_corpus/tune_fams_ground_truth/'
grd_truth_paths  = {fam: None for fam in tune_families}
for fam in tune_families:
    grd_truth_path = grd_truth_basepath + fam # 'tune_family_name' variable is defined in above code cell.
    assert os.path.isfile(grd_truth_path)
    print(f"Ground truth path for {fam[:-4]}:")
    print(grd_truth_path)
    grd_truth_paths[fam] = grd_truth_path


def format_input_data(in_path):
    """For a given tune family csv file located at 'in_path', this function extracts and lists all tune titles."""
    if 'precision' not in in_path:
        data = pd.read_csv(in_path)
        return data['title'].tolist()

# Execute above function and print sample output:
grd_truth = {}
for fam in grd_truth_paths:
    grd_truth[fam] = format_input_data(grd_truth_paths[fam])
    print(f"Ground truth data for {fam}:")
    for title in grd_truth[fam]:
        print(title)
    print('\n')

Ground truth path for jennys_welcome_to_charlie:
/Users/dannydiamond/NUIG/Polifonia/thesession/test_corpus/tune_fams_ground_truth/jennys_welcome_to_charlie.csv
Ground truth path for drowsy_maggie:
/Users/dannydiamond/NUIG/Polifonia/thesession/test_corpus/tune_fams_ground_truth/drowsy_maggie.csv
Ground truth path for gilderoy:
/Users/dannydiamond/NUIG/Polifonia/thesession/test_corpus/tune_fams_ground_truth/gilderoy.csv
Ground truth path for greigs_pipes:
/Users/dannydiamond/NUIG/Polifonia/thesession/test_corpus/tune_fams_ground_truth/greigs_pipes.csv
Ground truth path for blackbird:
/Users/dannydiamond/NUIG/Polifonia/thesession/test_corpus/tune_fams_ground_truth/blackbird.csv
Ground truth path for hob_or_nob:
/Users/dannydiamond/NUIG/Polifonia/thesession/test_corpus/tune_fams_ground_truth/hob_or_nob.csv
Ground truth path for osullivans_march:
/Users/dannydiamond/NUIG/Polifonia/thesession/test_corpus/tune_fams_ground_truth/osullivans_march.csv
Ground truth path for road_to_lisdoonvarna:


# Side-task:
# Update all similarity csv files, adding a Boolean column to improve readability and track tune family membership:

In [4]:
# TODO: revise output formatting from similarity search script to render below block obsolete
# NOTE: Do not re-run multiple times on same input data.

print("Updating similarity csv files...")
for fam in concatenated_inpaths:
    for path in concatenated_inpaths[fam]:
        print(path)
        similarity_data = pd.read_csv(path)

        # If reformatting is necessary, toggle this block

        if 'Unnamed: 0' in similarity_data.columns:
            if similarity_data['Unnamed: 0'].dtype == 'int64':
                similarity_data.pop('Unnamed: 0')
            elif similarity_data['Unnamed: 0'].dtype == 'object' or str:
                similarity_data.rename(columns={'Unnamed: 0': 'title'}, inplace=True)
        if 'count]' in similarity_data.columns:
            similarity_data.rename(columns={'count]': 'count'}, inplace=True)
        if 'presence]' in similarity_data.columns:
            similarity_data.rename(columns={'presence]': 'presence'}, inplace=True)
        if '_cos.csv' in path:
            similarity_data.rename({similarity_data.columns[1]: "Cosine similarity"}, axis=1, inplace=True)
        if 'tfidf' in path:
            similarity_data.rename({similarity_data.columns[1]: "Cosine similarity"}, axis=1, inplace=True)


        similarity_data['family member'] = similarity_data['title'].isin(grd_truth[fam])
        #     # print(similarity_data.head())
        similarity_data.to_csv(path)

print("Similarity csv files successfully updated.")





Updating similarity csv files...
/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/structural_results/incipit__lev.csv
/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/structural_results/incipit_cadence_cos.csv
/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/structural_results/incipit_cadence_lev.csv
/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/structural_results/incipit__cos.csv
/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4/jennys_welcome_to_charlie.csv/LongNoteThe9850/tfidf_results/tfidf_vector_cos_similarity.csv
/Users/dannydiamond/NUIG/Polifonia/thesession/similarity_results/similarity_experiment_4

KeyboardInterrupt: 

# Accuracy calculations

In [ ]:
# Calculate accuracy stats

labels = {
    'incipit__lev': 'Incipit -- Levenshtein distance',
    'incipit_cadence_cos': 'Incipit & cadence -- Cosine similarity',
    'incipit_cadence_lev': 'Incipit & cadence -- Levenshtein distance',
    'incipit__cos': 'Incipit -- Cosine similarity',
    'tfidf_vector_cos_similarity': 'TFIDF vector cosine similarity',
    'count_normalized': 'FONN count (normalized)',
    'presence': 'FONN presence',
    'count': 'FONN count',
    'presence_normalized': 'FONN presence (normalized)'
}

print("Calculating accuracy statistics...")
accuracy_results = []
for fam in concatenated_inpaths:
    for path in concatenated_inpaths[fam]:
        similarity_data = pd.read_csv(path)
        title = path.split('/')[-3]
        family = path.split("/")[-4][:-4]
        print(family)
        print(title)

        metric = ''
        for l in labels:
            if path.split('/')[-1][:-4] == l:
                metric = labels[l]
                print(metric)


        filtered_1 = similarity_data[similarity_data['family member'] == True]
        if not filtered_1.empty:
            filter_cutoff_idx = int(filtered_1.index[-1])
            print(f"Cutoff index: {filter_cutoff_idx}")
            filtered_2 = similarity_data.iloc[: filter_cutoff_idx + 1, ]
            thresh = np.NaN if filtered_2.empty else filtered_2.iloc[-1, 2].round(3)
            print(f'Threshold value: {thresh}')
            percentage_accuracy = filtered_2['family member'].value_counts(normalize=True, sort=False, dropna=False)
            if percentage_accuracy.empty:
                percentage_accuracy = pd.Series({True: 0, False: 1})

            idx = percentage_accuracy.index
            if True in idx:
                t = percentage_accuracy.at[True].round(3) * 100
        else:
            t = 0
            filter_cutoff_idx = np.NaN
            thresh = np.NaN

        print(f"Percentage accuracy: {t}")
        res = [family, title, metric, filter_cutoff_idx, thresh, t]
        accuracy_results.append(res)

cols = ['family', 'title', 'metric', 'cutoff threshold idx', 'cutoff threshold value', '% accuracy']
output = pd.DataFrame(accuracy_results, columns=cols)
output['% accuracy'] = output['% accuracy'].round(2)
output['cutoff threshold idx'] = output['cutoff threshold idx'].round(0)
print(output.head())
output.to_csv(root_dir + 'accuracy.csv')

# Add average accuracy numbers

mean_accuracy_results = pd.DataFrame()
for col in 'cutoff threshold idx', 'cutoff threshold value', '% accuracy':
    mean_accuracy_results['mean ' + col] = output.groupby('metric')[col].mean().round(4)
print(mean_accuracy_results.head())
mean_accuracy_results.to_csv(root_dir + 'mean_accuracy.csv')





# Run the precision ranking:

In [ ]:
# from IPython.display import display
#
# def precision_at_k(actual, measured, k):
#     """Simple precision at k implementation"""
#     grd_truth = set(actual)
#     predictions = set(measured[:k])
#     return round(len(predictions.intersection(grd_truth)) / min(k, len(actual)), 2)
#
# # set k values for calculations
# k_vals = [1, 5, 10, 20]
# # 'labels' variable will be used as an index column for csv and graph outputs:
# labels = {
#     'incipit__lev': 'Incipit -- Levenshtein distance',
#     'incipit_cadence_cos': 'Incipit & cadence -- Cosine similarity',
#     'incipit_cadence_lev': 'Incipit & cadence -- Levenshtein distance',
#     'incipit__cos': 'Incipit -- Cosine similarity',
#     'tfidf_vector_cos_similarity': 'TFIDF vector cosine similarity',
#     'count_normalized': 'FONN count (normalized)',
#     'presence': 'FONN presence',
#     'count': 'FONN count',
#     'presence_normalized': 'FONN presence (normalized)'
# }
#
# # 'format_input_data() function is defined above in code block 3
# # for tune_data in input_filepaths:
# #     input_data = [format_input_data(path) for path in tune_data if 'precision' not in path]
#
# # dict to store similarity results for all tunes, all metrics within each tune fam
# similarity_results = {}
# for fam in concatenated_inpaths:
#     res = []
#     for path in concatenated_inpaths[fam]:
#         if 'precision' not in path:
#             data = format_input_data(path)
#             res.append(data)
#     similarity_results[fam] = res
#
#
# # Reformat filenames for use in output csv paths
# results_filenames = {}
# for fam in concatenated_inpaths:
#     names = []
#     for path in concatenated_inpaths[fam]:
#         filename = path.split("/")[-1][:-4]
#         tune_title = path.split("/")[-3]
#         name = tune_title + "_" + filename
#         names.append(name)
#     results_filenames[fam] = names
#
# # print(results_filenames)
#
# # run and save p@k and ap@k calculations:
# precision = {}
# for fam in similarity_results:
#     grd_truth_data = grd_truth[fam]
#     res = []
#     for similarity_data in similarity_results[fam]:
#         # print("Ground truth:")
#         # print(grd_truth_data)
#         # print('\n')
#         # print("Similarity results:")
#         # print(similarity_data)
#         # print('\n')
#         patk = [round(precision_at_k(grd_truth_data, similarity_data, k), 2) for k in k_vals]
#         average_patk = [round(ml_metrics.apk(grd_truth_data, similarity_data, k), 2) for k in k_vals]
#         res.append(patk + average_patk)
#
#     precision[fam] = res
#     # we have a list of lists
#     # top list corresponds to each tune in fam
#     # lower lists are the 8 similarity metrics' results
#
#
# def reformat_idx(idx, mapper_dict):
#
#     split_name = idx.split("_")
#     suffix = '_'.join(split_name[1:])
#     row_name = split_name[0] + ": " + mapper_dict[suffix]
#     return row_name
#
#
# # format, shape and print results data:
# conc_results = []
# for fam in results_filenames:
#     output_mapper = dict(zip(results_filenames[fam], precision[fam]))
#     results = pd.DataFrame(output_mapper).T
#     results.columns = [[f'precision @ {k}' for k in k_vals] + [f'AP @ {k}' for k in k_vals] + [' threshold', ' % accuracy']]
#     indices  = results.index.to_series()
#     reformatted_indices = indices.apply(reformat_idx, args=[labels, fam])
#     results.set_index(reformatted_indices.astype(str), inplace=True, drop=True)
#     results.index = results.index.str.split(": ",expand=True)
#     results.reset_index(inplace=True, drop=False)
#     results.rename(columns={'level_0': 'title', 'level_1': 'metric'}, inplace=True)
#     outpath = f'{root_dir}{fam}/{fam[:-4]}_precision.csv'
#     conc_results.append(results)
#     results.to_csv(outpath)





In [ ]:
# conc_results_table = pd.concat(conc_results, ignore_index=True)
# conc_results_table.columns = conc_results_table.columns.map(''.join)
# print(conc_results_table.head())
# print(conc_results_table.info())
# print(conc_results_table.columns)
#
# corpus_level_map_results  = pd.DataFrame()
# for col in 'AP @ 1', 'AP @ 5', 'AP @ 10', ' threshold', ' % accuracy':
#     corpus_level_map_results['m' + col] = conc_results_table.groupby('metric')[col].mean().round(4)
# print(corpus_level_map_results)
# corpus_level_map_results.to_csv(root_dir + 'mean_average_precision.csv')

# Add mAP@k values to tune family tables:

In [ ]:
# # Open tune-family level precision results:
# precision_results_paths = [path for path in glob.iglob(os.path.join(root_dir, "**", "*.csv"), recursive=True) if "precision" in path and not 'mean' in path]
# print(precision_results_paths)
#
# tune_family_res_tables = []
# for path in precision_results_paths:
#     fam = path.split("/")[-1][:-14]
#     data = pd.read_csv(path, index_col=0)
#     res = pd.DataFrame()
#     for col in 'AP @ 1', 'AP @ 5', 'AP @ 10', ' threshold', ' % accuracy':
#         res['m' + col] = data.groupby('metric')[col].mean().round(4)
#     out_path = path[:-13] + 'mean_average_precision.csv'
#     res.reset_index(inplace=True)
#     res.insert(0, 'family', fam)
#     tune_family_res_tables.append(res)
#     res.to_csv(out_path)
#
# all_tune_families = pd.concat(tune_family_res_tables)
# all_tune_families.to_csv(root_dir + 'mean_average_precision_per_tune_family.csv')
# print(all_tune_families.head())
# print(all_tune_families.info())
#
# corpus_level_map_results_per_tune_fam  = pd.DataFrame()
# for col in 'mAP @ 1', 'mAP @ 5', 'mAP @ 10', 'm threshold', 'm % accuracy':
#     corpus_level_map_results_per_tune_fam['Average ' + col] = all_tune_families.groupby('metric')[col].mean().round(4)
# print(corpus_level_map_results_per_tune_fam)
# corpus_level_map_results_per_tune_fam.to_csv(root_dir + 'corpus_results_per_tune_family.csv')


In [ ]:
# # Visualize results:
#
# # def visualize_results(input_path, target_col, title, viz_outpath):
# #     input_data = pd.read_csv(input_path, index_col="metric")
# #     print(input_data.head())
# #     viz = input_data.plot.bar(y=target_col, figsize=(12, 8), title=title)
# #     plt.title(f'{title} for {tune_family_name}')
# #     plt.savefig(fname=f"{viz_outpath}.png", bbox_inches="tight", dpi=360, facecolor='white', transparent=False)
# #     plt.show()
# input_data = corpus_level_map_results_per_tune_fam
#
# x = input_data.index
# x_axis = np.arange(len(x))
# mAPat1 = input_data['Average mAP @ 1']
# mAPat5 = input_data['Average mAP @ 5']
# mAPat10 = input_data['Average mAP @ 10']
#
# plt.bar(x_axis - 0.2, mAPat1, 0.4, label = 'mAP @ 1')
# plt.bar(x_axis, mAPat5, 0.4, label = 'mAP @ 5')
# plt.bar(x_axis + 0.2, mAPat10, 0.4, label = 'mAP @ 10')
# plt.xticks(x_axis, x)
# plt.xticks(rotation=45, ha="right")
# plt.xlabel("Metrics")
# plt.ylabel("Mean average precision")
# plt.title("Mean average precision (averaged per tune family) for The Session corpus:\nComparing 9 similarity metrics on accent-level input data.")
# plt.legend()
# graph_out_path = root_dir + 'mean_average_precision_per_tune_family'
# plt.savefig(fname=f"{graph_out_path}.png", bbox_inches="tight", dpi=360, facecolor='white', transparent=False)
# plt.show()





In [ ]:
# TODO: Revise tune family member assignment with expert input (and informed by results -- can we find previously-unknown tune family members?)
# TODO: Should search term tune be removed from results?
# TODO: Explore local alignment -- LCSS & Needleman Wunch in particular
# TODO: Explore structural heuristics / metrics and structural algorithms (entropy; pattern occurrences/onsets, replication of Seán Doherty's work)
# TODO: Combine structural and melodic results, and/or multiple metrics at different levels. EG: Incipit @ note-level plus TFIDF cosine @ accent plus entropy
# TODO: Explore ML options to implement in parallel?
# TODO: Add custom Levenshtein weights, try shorter incipits.

# Clean up this notebook and refactor corpus_processing_tools.py for next deliverable.
# Write up and potentially prep for publication?
# Image to aid meeting on 1st Dec.

# Additional corpora: Meertens, Essen, ITMA ...
# PDP, GRC, course registration, & fees; exchange within Polifonia; Marie Curie

# Results observations:
# Best performing metrics perform better than they did in the last experiment, worse-performing metrics performing worse than in previous experiment.
# Decline of motific metrics may be due to excess of similar patterns due to normalising effect of switch from chromatic to diatonic input data. This may be worth further exploration: try chromatic input data with a higher Levenshtein threshold and/or custom weightings alongside diatonic with low threshold and custom weightings.
# ITM: Effectiveness of incipit plus cadence method somewhat supports MÓS's paper (though more effective so far at note rather than accent-level). Try again with shorter accents at note-level in particular, with custom weightings and possibly an adjusted distance threshold.
# Headline [for me]: in the accent-level TF-IDF results we have a unique high-performing metric combining ITM knowledge plus classical ML. Although performance is not best in class, it is extremely fast and run locally on large datasets in minutes. In this respect it is best in class.